Once installed, you need to load the `pytorch_memlab` IPython extensions:

In [1]:
%load_ext pytorch_memlab

One magic is provided, `mlrun` which can act either as a line magic `%mlrun`, or as a cell magic `%%mlrun`

In [2]:
%%mlrun?

Docstring:
::

  %mlrun [--function FUNC] [-r] [-T OUTPUT] [-g GPU_ID] [-q]
             [statement [statement ...]]

Execute a statement/cell under the PyTorch Memlab profiler to collect CUDA memory
allocation information on a per-line basis.

positional arguments:
  statement             Code to run under profiler. You can omit this in cell
                        magic mode.

optional arguments:
  --function FUNC, -f FUNC
                        Function to profile. Can be specified multiple times
  -r, --return-profiler
                        Return LineProfiler object for introspection
  -T OUTPUT, --dump-profile OUTPUT
                        Dump text profile output to file
  -g GPU_ID, --gpu GPU_ID
                        Profile memory usage of GPU ID
  -q, --quiet           Don't print out profile results
File:      ~/pytorch_memlab/pytorch_memlab/extension.py


First we need some torch code to profile:

In [3]:
import torch

def x():
    torch.nn.Linear(100, 100).cuda()
    
def y(gpu=0):
    torch.nn.Linear(1000, 100).cuda(device=gpu)

We can profile multiple functions at the same type by repeatedly specifying `-f`

In [4]:
%%mlrun -f x -f y

x()
y()

File: <ipython-input-3-319e964b59a0>
Function: x at line 3

Line # Max usage   Peak usage diff max diff peak  Line Contents
     3                                           def x():
     4     0.00B        2.00M    0.00B    2.00M      torch.nn.Linear(100, 100).cuda()

File: <ipython-input-3-319e964b59a0>
Function: y at line 6

Line # Max usage   Peak usage diff max diff peak  Line Contents
     6                                           def y(gpu=0):
     7     0.00B        2.00M    0.00B    2.00M      torch.nn.Linear(1000, 100).cuda(device=gpu)



You can alos profile with the `%mlrun` line magic

In [5]:
def z():
    torch.nn.Linear(100, 100).cuda()
%mlrun -f z z()

File: <ipython-input-5-b6fdc6192c4c>
Function: z at line 1

Line # Max usage   Peak usage diff max diff peak  Line Contents
     1                                           def z():
     2     0.00B        2.00M    0.00B    2.00M      torch.nn.Linear(100, 100).cuda()



You can specify which GPU you wish to profile using `-g`:

In [6]:
%%mlrun -f x -f y -g 1 y

x()
y(gpu=1)

File: <ipython-input-3-319e964b59a0>
Function: x at line 3

Line # Max usage   Peak usage diff max diff peak  Line Contents
     3                                           def x():
     4     0.00B        0.00B    0.00B    0.00B      torch.nn.Linear(100, 100).cuda()

File: <ipython-input-3-319e964b59a0>
Function: y at line 6

Line # Max usage   Peak usage diff max diff peak  Line Contents
     6                                           def y(gpu=0):
     7     0.00B        2.00M    0.00B    2.00M      torch.nn.Linear(1000, 100).cuda(device=gpu)



You can get a handle on the `LineProfiler` object using `-r`

In [7]:
profiler = %mlrun -q -r -f x x()
profiler.code_map

{<code object x at 0x7f8dae4f6150, file "<ipython-input-3-319e964b59a0>", line 3>: {'line_stat': defaultdict(list,
              {-1: [(0, 0)], 4: [(0, 2097152)]}),
  'func': <function __main__.x()>,
  'func_name': 'x',
  'source_code': (['def x():\n', '    torch.nn.Linear(100, 100).cuda()\n'], 3),
  'last_lineno': 0}}

You can dump stats out to a file using `-T`:

In [8]:
%mlrun -q -T profile.log -f x x()

In [9]:
!head profile.log

File: <ipython-input-3-319e964b59a0>
Function: x at line 3

Line # Max usage   Peak usage diff max diff peak  Line Contents
     3                                           def x():
     4     0.00B        2.00M    0.00B    2.00M      torch.nn.Linear(100, 100).cuda()

